# Captioning Model Builder

### [과정]
### **1) 캡션 텍스트 전처리 및 임베딩 준비**<br>
텍스트를 단어별로 나누는 토큰화 작업을 하고, 단어들을 숫자로 변환한 후, 이를 임베딩 벡터로 변환할 준비 해두기<br>
-> 추후 LSTM에서 사용 예정

In [1]:
import os
import tensorflow as tf
import psutil

In [2]:
# GPU 체크
gpu_devices = tf.config.list_physical_devices('GPU')
if len(gpu_devices) > 0:
    print(f"Connected to GPU: {gpu_devices}")
else:
    print("Not connected to a GPU")

!nvidia-smi

Connected to GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Mon Oct 14 21:03:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX450         WDDM  |   00000000:2D:00.0 Off |                  N/A |
| N/A   44C    P8             N/A / ERR!  |       0MiB /   2048MiB |      0%      Default |
|                                         |            

In [3]:
# RAM 체크
ram_gb = psutil.virtual_memory().total / 1e9
print('Your system has {:.1f} gigabytes of available RAM'.format(ram_gb))
print('-------------------------------------------')

Your system has 8.2 gigabytes of available RAM
-------------------------------------------


In [4]:
# TensorFlow 버전 확인
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.10.0


# 데이터 읽기

### AI Hub MSCOCO 한국어 이미지 설명 데이터셋

[기본 정보]<br>
- 전체 : 123,287
- train data : 82,783
- validation data : 40,504

**[문제] 데이터셋에 이미지가 포함이 안 되어 있음**<br>
-> 어떻게 처리하지?<br>
일단 데이터 보니까 안에 이미지의 url 주소가 있음 <- 이거로 다운받아 와야 할듯 ,,

In [5]:
import os
import json

In [6]:
# 데이터 경로 설정
data_path = os.path.join("data", "MSCOCO_train_val_Korean.json")

In [7]:
# JSON 파일 열기
with open(data_path, 'r', encoding='UTF8') as f:
    json_data = json.load(f)

In [8]:
# 데이터 확인
print(f"Total entries in the dataset: {len(json_data)}")

Total entries in the dataset: 123287


In [9]:
# 첫 번째 데이터 확인
json_data[0]

{'file_path': 'val2014/COCO_val2014_000000391895.jpg',
 'captions': ['A man with a red helmet on a small moped on a dirt road. ',
  'Man riding a motor bike on a dirt road on the countryside.',
  'A man riding on the back of a motorcycle.',
  'A dirt path with a young person on a motor bike rests to the foreground of a verdant area with a bridge and a background of cloud-wreathed mountains. ',
  'A man in a red shirt and a red hat is on a motorcycle on a hill side.'],
 'id': 391895,
 'caption_ko': ['빨간 헬멧을 쓴 남자가 작은 모터 달린 비포장 도로를 달려 있다.',
  '시골의 비포장 도로에서 오토바이를 타는 남자',
  '오토바이 뒤에 탄 남자',
  '오토바이 위에 젊은이가 탄 비포장 도로는 다리가 있는 초록빛 지역의 전경과 구름 낀 산의 배경이 있다.',
  '빨간 셔츠와 빨간 모자를 쓴 남자가 언덕 쪽 오토바이 위에 있다.']}

In [10]:
# 첫 번째 데이터의 파일 경로 확인
print(f"File path of the first entry: {json_data[0]['file_path']}")

File path of the first entry: val2014/COCO_val2014_000000391895.jpg


In [11]:
# 'train2014'와 'val2014'로 나뉜 데이터의 개수 세기
train_data = [item for item in json_data if item['file_path'].startswith('train2014')]
val_data = [item for item in json_data if item['file_path'].startswith('val2014')]

print(f"Number of train samples: {len(train_data)}")
print(f"Number of val samples: {len(val_data)}")

Number of train samples: 82783
Number of val samples: 40504


### MSCOCO 이미지 처리

In [12]:
import os
import requests
import zipfile

In [13]:
# 이미지 저장을 위한 디렉토리 생성 (이미 존재하면 무시)
current_dir = os.getcwd()
images_dir = os.path.join(current_dir, 'images')  # 변경할 경로 설정

train_zip_path = os.path.join(images_dir, 'train2014.zip')
val_zip_path = os.path.join(images_dir, 'val2014.zip')

if not os.path.exists(images_dir):
    os.makedirs(images_dir)

In [14]:
# 경로 이동
os.chdir(images_dir)
print(f"Current directory: {os.getcwd()}")

Current directory: c:\Users\김소연\Desktop\soyeon\ShortClip-Caption-Generator\images


In [15]:
# 데이터셋 다운로드 함수
def download_file(url, destination):
    response = requests.get(url, stream=True)
    with open(destination, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

In [16]:
# train2014.zip 다운로드 여부 확인
if not os.path.exists(train_zip_path):
    print("Downloading train2014.zip...")
    download_file('http://images.cocodataset.org/zips/train2014.zip', train_zip_path)
else:
    print("train2014.zip already exists, skipping download.")

# val2014.zip 다운로드 여부 확인
if not os.path.exists(val_zip_path):
    print("Downloading val2014.zip...")
    download_file('http://images.cocodataset.org/zips/val2014.zip', val_zip_path)
else:
    print("val2014.zip already exists, skipping download.")


train2014.zip already exists, skipping download.
val2014.zip already exists, skipping download.


In [17]:
# 압축 해제 함수
def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

In [18]:
# train2014 압축 해제 여부 확인
train_images_dir = os.path.join(images_dir, 'train2014')
if not os.path.exists(train_images_dir):
    print("Extracting train2014.zip...")
    extract_zip(train_zip_path, images_dir)
else:
    print("train2014 already extracted, skipping extraction.")

# val2014 압축 해제 여부 확인
val_images_dir = os.path.join(images_dir, 'val2014')
if not os.path.exists(val_images_dir):
    print("Extracting val2014.zip...")
    extract_zip(val_zip_path, images_dir)
else:
    print("val2014 already extracted, skipping extraction.")

train2014 already extracted, skipping extraction.
val2014 already extracted, skipping extraction.


In [19]:
# 이미지 개수 확인
train_images_path = os.path.join(images_dir, 'train2014')
val_images_path = os.path.join(images_dir, 'val2014')

train_image_count = len(os.listdir(train_images_path))
val_image_count = len(os.listdir(val_images_path))

print(f"Number of images in train2014: {train_image_count}")
print(f"Number of images in val2014: {val_image_count}")

# 데이터셋 개수 확인
print(f"Total dataset size: {train_image_count + val_image_count}")

Number of images in train2014: 82783
Number of images in val2014: 40504
Total dataset size: 123287


# 데이터 전처리

### 불용어 정리

In [20]:
import os

In [21]:
# 불용어 파일들이 위치한 디렉토리 경로
os.chdir('..')
current_dir = os.getcwd()
input_dir = os.path.join(current_dir, 'stopwords')
print(current_dir)
output_file = 'combined_stopwords.txt'

c:\Users\김소연\Desktop\soyeon\ShortClip-Caption-Generator


In [22]:
# 불용어 파일이 이미 존재하는지 확인
if os.path.exists(output_file):
    print(f"{output_file} already exists. Skipping combining stopwords.")
else:
    print(f"{output_file} does not exist. Combining stopwords...")
    
    # 중복을 제거한 불용어들을 저장할 세트
    final_stopwords = set()

    # stopwords 폴더 내 모든 txt 파일을 읽어 중복 제거
    for filename in os.listdir(input_dir):
        if filename.endswith('.txt'):  # .txt 파일만 처리
            file_path = os.path.join(input_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                stopwords = f.readlines()
                # 각 파일에서 불용어를 추가하고 중복을 자동으로 제거
                final_stopwords.update(word.strip() for word in stopwords)

    # 중복이 제거된 불용어 리스트를 combined_stopwords.txt 파일에 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        for word in sorted(final_stopwords):  # 정렬하여 저장 (선택 사항)
            f.write(word + '\n')

    print(f"Combined stopwords saved to {output_file}.")

combined_stopwords.txt does not exist. Combining stopwords...
Combined stopwords saved to combined_stopwords.txt.


### 캡션 텍스트 전처리 및 토큰화

불용어들을 제거하고 의미가 있는 형태소만 남기기<br>
-> 의미가 있는 핵심 단어들만 담아서 정제된 상태로 핵심 정보만 남겨서 학습시키기

In [23]:
import pickle
import re
import os

from tqdm import tqdm
from konlpy.tag import Okt

In [24]:
json_data[0]['file_path'].split('/')[1]

'COCO_val2014_000000391895.jpg'

In [25]:
# 형태소 분석기 및 변수 초기화
okt = Okt()
image_caption_dict = dict()
sent_token = []
max_length = 0

In [26]:
# 불용어 파일 경로
current_dir = os.getcwd()

stopwords_file = os.path.join(current_dir, 'combined_stopwords.txt')
output_file = os.path.join(current_dir, 'image_caption_dict.pkl')  # 처리된 결과 파일 경로

In [30]:
# combined_stopwords.txt 파일에서 불용어 불러오기
with open(stopwords_file, "r", encoding="utf-8") as f:
    stopwords = set(line.strip() for line in f.readlines())  # 불용어를 set으로 저장

In [31]:
# 결과 파일이 이미 존재하는지 확인
if os.path.exists(output_file):
    print(f"{output_file} already exists. Skipping processing.")

    # 파일이 존재하면 데이터 로드
    with open(output_file, "rb") as f:
        image_caption_dict = pickle.load(f)

    # 로드된 데이터를 바탕으로 max_length 다시 계산
    for desc_list in image_caption_dict.values():
        for desc in desc_list:
            desc_words = desc.split()
            max_length = max(max_length, len(desc_words))

else:
    # 파일이 존재하지 않으면 데이터 처리
    print(f"{output_file} does not exist. Processing data...")

    # 이미지 및 캡션 처리
    for entry in tqdm(json_data):
        id = entry['file_path'].split('/')[1]  # jpg 파일 이름 추출
        descs = []

        for desc in entry['caption_ko']:
            # 전처리 및 형태소 분석
            desc = re.sub('[^가-힣 ]', '', desc)  # 한글 외 제거
            desc_words = [word for word in okt.morphs(desc, stem=True) if word not in stopwords]  # 형태소 분석 및 불용어 제거
            
            # 토큰 리스트 및 최대 길이 갱신
            sent_token.append(desc_words)
            max_length = max(max_length, len(desc_words))

            # 형태소 리스트를 하나의 문자열로 변환 후 저장
            descs.append(' '.join(desc_words))

        # 이미지 ID를 key로, 5개의 description 리스트를 value로 가진 dictionary 생성
        image_caption_dict[id] = descs

    # 결과 저장
    with open(output_file, "wb") as f:
        pickle.dump(image_caption_dict, f)
    print(f"{output_file} has been created and saved.")

c:\Users\김소연\Desktop\soyeon\ShortClip-Caption-Generator\image_caption_dict.pkl already exists. Skipping processing.


In [32]:
# 전체 고유 단어 추출
unique_words = {word for token_list in image_caption_dict.values() for token in token_list for word in token.split()}

# 결과 출력
print('max length:', max_length)  # 가장 긴 캡션 길이
print('number of {image-descs}:', len(image_caption_dict))  # 총 이미지-캡션 데이터 개수
print('img_desc_dict looks like:', list(image_caption_dict.items())[0])  # 예시 데이터 확인
print('total number of unique words:', len(unique_words))  # 고유 단어 수

max length: 39
number of {image-descs}: 123287
img_desc_dict looks like: ('COCO_val2014_000000391895.jpg', ['빨갛다 헬멧 쓸다 남자 작다 모터 달리다 비 포장 도로 달다', '시골 비 포장 도로 오토바이 남자', '오토바이 뒤 남자', '오토바이 위 젊은이 비 포장 도로 다리 초록빛 지역 전경 구름 끼다 산 배경', '빨갛다 셔츠 빨갛다 모자 쓸다 남자 언덕 쪽 오토바이 위'])
total number of unique words: 15496


조사, 어미 등과 같은 문법적 관계를 포현하는 형태소도 추가해주기<br>
-> 나중에 자연스러운 문장을 생성할 수 있도록

In [33]:
import os
import re
import pickle

from tqdm import tqdm
from konlpy.tag import Okt

In [34]:
# 형태소 분석기 및 변수 초기화
okt = Okt()
image_caption_dict = dict()
sent_token = []
max_length = 0

In [35]:
# 결과 파일 경로 설정
output_file = os.path.join(os.getcwd(), "image_caption_dict2.pkl")

In [36]:
# 결과 파일이 이미 존재하는지 확인
if os.path.exists(output_file):
    print(f"{output_file} already exists. Loading data...")

    # 파일이 존재하면 데이터 로드
    with open(output_file, "rb") as f:
        image_caption_dict = pickle.load(f)

    # 로드된 데이터를 바탕으로 max_length 다시 계산
    for desc_list in image_caption_dict.values():
        for desc in desc_list:
            desc_words = desc.split()
            max_length = max(max_length, len(desc_words))

else:
    print(f"{output_file} does not exist. Processing data...")

    # json_data에 대한 처리
    for entry in tqdm(json_data):
        # 이미지 파일 이름 추출
        id = entry['file_path'].split('/')[1]  # jpg 파일 이름 추출
        descs = []

        # 각 이미지에 대한 5개의 캡션 처리
        for desc in entry['caption_ko']:
            desc = re.sub('[^가-힣 ]', '', desc) # 전처리: 한글만 남기고 나머지 제거
            desc_words = okt.morphs(desc) # 형태소 분석 (의미형태소 + 기능형태소)

            sent_token.append(desc_words) # 임베딩용 토큰 리스트 추가

            # 최대 길이 업데이트
            max_length = max(max_length, len(desc_words))

            # 형태소 리스트를 공백으로 결합한 후 저장
            descs.append(' '.join(desc_words))

        # 이미지 이름을 key로, 5개의 전처리된 캡션 리스트를 value로 가진 dictionary 생성
        image_caption_dict[id] = descs

    # 처리 후 결과를 .pkl 파일로 저장
    with open(output_file, "wb") as f:
        pickle.dump(image_caption_dict, f)
    print(f"{output_file} has been created and saved.")


c:\Users\김소연\Desktop\soyeon\ShortClip-Caption-Generator\image_caption_dict2.pkl does not exist. Processing data...


100%|██████████| 123287/123287 [31:52<00:00, 64.46it/s]  


c:\Users\김소연\Desktop\soyeon\ShortClip-Caption-Generator\image_caption_dict2.pkl has been created and saved.


In [37]:
# 전체 고유 단어 추출
unique_words = set(word for token_list in image_caption_dict.values() for token in token_list for word in token.split())

# 결과 출력 (파일이 있든 없든 항상 실행)
print(f'max length: {max_length}')  # 가장 긴 캡션의 길이
print(f'number of image-descriptions: {len(image_caption_dict)}')  # 총 이미지-캡션 데이터 개수
print(f'img_desc_dict example: {list(image_caption_dict.items())[0]}')  # 첫 번째 데이터 예시
print(f'total number of unique words: {len(unique_words)}')  # 고유 단어 수

max length: 55
number of image-descriptions: 123287
img_desc_dict example: ('COCO_val2014_000000391895.jpg', ['빨간 헬멧 을 쓴 남자 가 작은 모터 달린 비 포장 도로 를 달려 있다', '시골 의 비 포장 도로 에서 오토바이 를 타는 남자', '오토바이 뒤 에 탄 남자', '오토바이 위 에 젊은이 가 탄 비 포장 도로 는 다리 가 있는 초록빛 지역 의 전경 과 구름 낀 산 의 배경 이 있다', '빨간 셔츠 와 빨간 모자 를 쓴 남자 가 언덕 쪽 오토바이 위 에 있다'])
total number of unique words: 24719


In [38]:
with open("image_caption_dict2.pkl", "rb") as f:
    img_desc_dict = pickle.load(f)

In [39]:
for x in sent_token[:5]:
  print(x)

['빨간', '헬멧', '을', '쓴', '남자', '가', '작은', '모터', '달린', '비', '포장', '도로', '를', '달려', '있다']
['시골', '의', '비', '포장', '도로', '에서', '오토바이', '를', '타는', '남자']
['오토바이', '뒤', '에', '탄', '남자']
['오토바이', '위', '에', '젊은이', '가', '탄', '비', '포장', '도로', '는', '다리', '가', '있는', '초록빛', '지역', '의', '전경', '과', '구름', '낀', '산', '의', '배경', '이', '있다']
['빨간', '셔츠', '와', '빨간', '모자', '를', '쓴', '남자', '가', '언덕', '쪽', '오토바이', '위', '에', '있다']
